In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate


In [2]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True




### 1. 设置 os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com' ，这样做具体改变了什么？ 
 
 os.environ  是Python中用于访问和操作环境变量的对象。 HF_ENDPOINT  是Hugging Face相关库（如  datasets 、 transformers  等）用来确定从哪里获取模型、数据集等资源的环境变量。当设置  os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'  后，Hugging Face的相关库在请求资源时，会从指定的镜像站点  https://hf-mirror.com  而不是官方默认的源去获取数据。这可以在一些情况下提高访问速度、绕过网络限制等。

### 为什么要设置HF_ENDPOINT='https://hf-mirror.com'而非直接使用官方源？ 
 网络限制和访问问题：在某些网络环境中，官方源可能被限制访问，通过设置镜像站点可以绕过这些限制，正常获取资源。
 
 提高下载速度：镜像站点可能在地理位置上更接近用户，或者有更好的网络带宽，从而提高下载模型和数据集的速度。
 
 减轻官方源压力：使用镜像站点可以分散流量，减轻Hugging Face官方源的负载，有助于整个生态系统的稳定运行。



## dataset = datasets.load_dataset("bentrevett/multi30k") 这行代码具体完成了什么操作？

这行代码使用了Hugging Face的  datasets  库来加载名为  "bentrevett/multi30k"  的数据集。具体来说，它会： 
 检查本地是否已经缓存了该数据集。如果已经缓存，则直接从本地加载。
 如果本地没有缓存，它会根据  HF_ENDPOINT  环境变量（如果已设置）指定的地址，从远程获取该数据集，并下载到本地缓存目录。
 最后，将加载好的数据集对象赋值给变量  dataset ，后续可以对该数据集进行各种操作，如查看数据样本、进行数据预处理等。 multi30k  数据集是一个用于机器翻译和多模态研究的数据集，包含图像描述对（源语言和目标语言）。


In [3]:
import os
from datasets import load_dataset

# 1. 基础修改方式
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 2. 加载数据集（自动走镜像）
dataset = load_dataset("multi30k")


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [5]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

In [6]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

In [7]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [8]:
string = "What a lovely day it is today!"

[token.text for token in en_nlp.tokenizer(string)]

['What', 'a', 'lovely', 'day', 'it', 'is', 'today', '!']

### 函数  tokenize_example 

其功能是对包含英文和德文文本的样本进行分词、文本处理（如转换为小写），并添加开始和结束标记。

In [9]:
def tokenize_example(example, en_nlp, de_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}


### sos和eos的含义 
sos_token = "<sos>"  中的  "sos"  是 "Start of Sentence" 的缩写，即句子开始标记。在自然语言处理中，尤其是在序列到序列（如机器翻译）等任务里，模型需要知道一个句子从哪里开始。因此，会在每个句子的开头添加这个特殊的标记，这样模型就能识别句子的起始位置。
 
eos_token = "<eos>"  中的  "eos"  是 "End of Sentence" 的缩写，即句子结束标记。类似地，模型需要知道一个句子在何处结束。在处理完一个句子的所有单词后，添加这个特殊标记，让模型明确句子的终止。

### map函数的作用
 map  函数的作用是对数据集中的每个样本应用指定的函数

In [10]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "de_nlp": de_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:

train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

In [12]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)


In [13]:
en_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', 'a', '.', 'in', 'the', 'on', 'man']

In [14]:
de_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', '.', 'ein', 'einem', 'in', 'eine', ',']

In [15]:
en_vocab["the"]

7

In [16]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [17]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)


In [18]:
tokens = ["i", "love", "watching", "crime", "shows"]
en_vocab.lookup_indices(tokens)

[956, 2169, 173, 0, 821]


## 词汇表转换中"crime"变成<unk>的原因解析

crime 不在 en_vocab 的词表中，导致被映射到 <unk>。

In [19]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'love', 'watching', '<unk>', 'shows']

### 两个单元格中代码的作用
将原始文本中的单词（tokens）转换为对应的数值索引，以便模型能够处理。

In [20]:
def numericalize_example(example, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

In [21]:
fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:

train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>'],
 'en_ids': [2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3],
 'de_ids': [2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]}

In [23]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [24]:

train_data[0]

{'en_ids': tensor([   2,   16,   24,   15,   25,  778,   17,   57,   80,  202, 1312,    5,
            3]),
 'de_ids': tensor([   2,   18,   26,  253,   30,   84,   20,   88,    7,   15,  110, 7647,
         3171,    4,    3]),
 'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}


## 数据处理函数解析

### `get_collate_fn` 函数

**作用**：
创建用于处理批次数据的整理函数(collate function)，主要实现：
1. 从批次中提取英语和德语的ID序列
2. 对不等长序列进行填充对齐
3. 返回整理后的批次数据


### `get_data_loader` 函数

**作用**：
创建PyTorch数据加载器(DataLoader)，主要功能：
1. 配置数据加载参数
2. 应用自定义的整理函数
3. 控制数据加载行为



In [25]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

In [26]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader
get_collate_fn(pad_index)

<function __main__.get_collate_fn.<locals>.collate_fn(batch)>

In [27]:

batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)


### 解释下方单元格中Encoder类的代码。

​​1️⃣ 类定义与初始化​​

'''class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)'''

​​组件作用​​：

​​nn.Embedding​​:

将输入的离散词索引（如 [5, 3, 2]）映射为连续的词向量。

​​参数​​：

input_dim: 词汇表大小（如 10,000 个单词）。

embedding_dim: 词向量维度（如 256 维）。

​​nn.LSTM​​:

长短期记忆网络，处理序列数据并捕捉长期依赖。

​​参数​​：

embedding_dim: 输入维度（需与 Embedding 层输出一致）。

hidden_dim: LSTM 隐藏层维度（如 512 维）。

n_layers: LSTM 的堆叠层数（如 2 层）。

dropout: 层间 dropout 概率（仅在多层 LSTM 中生效）。

​​nn.Dropout​​:

在训练时随机丢弃部分神经元，防止过拟合。

2️⃣ 前向传播 (forward)​

'''def forward(self, src):
    embedded = self.dropout(self.embedding(src))  # [src_len, batch_size, emb_dim]
    outputs, (hidden, cell) = self.rnn(embedded)
    return hidden, cell'''
    
输入与输出​​：

​​输入 src​​:

形状：[src_len, batch_size]

（例如：句子长度为 10，批次大小为 32 → [10, 32]）

每个元素是词索引（如 [5, 3, 2]）。

​​输出​​:

hidden: 最后一层的隐藏状态，形状 [n_layers, batch_size, hidden_dim]。

cell: 最后一层的细胞状态，形状与 hidden 相同。

​​关键步骤​​：

​​词嵌入与 Dropout​​:

self.embedding(src) 将词索引转换为词向量 → 形状 [src_len, batch_size, embedding_dim]。

self.dropout 对词向量进行随机置零，增强泛化性。

​​LSTM 处理​​:

输入 embedded 经过 LSTM 层，输出：

outputs: 所有时间步的隐藏状态（通常用不到，因为只保留最后一层）。

hidden: 最后一层的隐藏状态（用于传递给 Decoder）。

cell: 最后一层的细胞状态（LSTM 的内部记忆）。

​​3️⃣ 设计特点​​

​​单向 LSTM​​:

默认仅使用单向 LSTM（bidirectional=False），若需双向需修改 nn.LSTM 参数。

​​多层 LSTM​​:

通过 n_layers 堆叠多个 LSTM 层，深层网络可捕获更复杂的模式。

​​Dropout 策略​​:

仅在 LSTM 层间应用 dropout（通过 nn.LSTM 的 dropout 参数），最后一层后额外添加 self.dropout。

In [28]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

### 描述Decoder的工作流程。

1️⃣ 核心目标：序列生成​​

Decoder 的作用是根据 Encoder 编码的上下文信息（hidden 和 cell 状态），​​自回归地生成目标序列​​（如翻译后的句子）。每次处理一个时间步，逐步预测下一个词。

​​2️⃣ 输入与输出​​

​​输入​​：

input: 上一个时间步生成的词索引（训练时为真实标签，推理时为模型预测的词）。

形状：[batch_size]（例如：批次中每个样本的上一个词索引）。

hidden 和 cell: Encoder 传递的初始状态（或前一步的隐藏状态）。

形状：[n_layers, batch_size, hidden_dim]。

​​输出​​：

prediction: 当前时间步的预测词概率分布。

形状：[batch_size, output_dim]（例如：德语词汇表大小的向量）。

更新后的 hidden 和 cell 状态，供下一个时间步使用。

​​3️⃣ 前向传播 (forward) 流程​​

​​步骤 1：输入形状调整​​

input = input.unsqueeze(0)  # 从 [batch_size] → [1, batch_size]

​​目的​​：将单步输入转换为 LSTM 需要的序列格式 [seq_len, batch_size]（此处 seq_len=1）。

​​步骤 2：词嵌入与 Dropout​​

embedded = self.dropout(self.embedding(input))  # [1, batch_size, embedding_dim]

​​词嵌入​​：将离散词索引映射为连续向量。

​​Dropout​​：随机丢弃部分词向量维度，防止过拟合。

​​步骤 3：LSTM 处理​​

output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

​​输入​​：

embedded: 当前时间步的词向量。

hidden 和 cell: 前一步的隐藏状态和细胞状态。

​​输出​​：

output: 当前时间步的隐藏状态（实际未直接使用）。

hidden 和 cell: 更新后的状态，传递给下一步。

​​步骤 4：预测生成​​

prediction = self.fc_out(output.squeeze(0))  # 从 [1, batch_size, hidden_dim] → [batch_size, output_dim]

​​Squeeze​​: 移除多余的序列长度维度（seq_len=1）。

​​全连接层​​：将 LSTM 输出映射到目标词汇表空间（如德语词汇表大小）。

In [29]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

### 解释下方单元格中Seq2Seq类的代码。

​​1️⃣ 类定义与初始化​​

'''class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        # 确保编码器和解码器的隐藏层维度、层数一致
        assert encoder.hidden_dim == decoder.hidden_dim, "隐藏层维度必须一致！"
        assert encoder.n_layers == decoder.n_layers, "层数必须一致！"'''

​​关键设计点​​：

​​编码器-解码器架构​​：encoder 处理输入序列，decoder 生成输出序列。

​​设备管理​​：确保张量在 CPU/GPU 上正确分配。

​​2️⃣ forward 函数流程​​

​​输入​​：

src: 输入序列（源语言），形状 [src_len, batch_size]。

trg: 目标序列（目标语言），形状 [trg_len, batch_size]（含 <sos> 和 <eos>）。

teacher_forcing_ratio: 使用真实标签作为下一步输入的概率（如 0.75）。

​​输出​​：

outputs: 所有时间步的预测结果，形状 [trg_len, batch_size, trg_vocab_size]。

​​步骤解析​​：

​​初始化​​：

'''batch_size = trg.shape[1]
trg_length = trg.shape[0]
outputs = torch.zeros(trg_length, batch_size, self.decoder.output_dim).to(self.device)
hidden, cell = self.encoder(src)  # 编码器生成初始状态
input = trg[0, :]  # 第一个输入是 <sos> 符号'''

​​循环生成序列​​（从时间步 1 到 trg_length-1）：

'''for t in range(1, trg_length):
    output, hidden, cell = self.decoder(input, hidden, cell)  # 解码器单步推理
    outputs[t] = output  # 保存当前时间步的预测
    
    # Teacher Forcing 决策
    teacher_force = random.random() < teacher_forcing_ratio
    top1 = output.argmax(1)  # 当前预测的词
    input = trg[t] if teacher_force else top1  # 输入下一时间步的真实词或预测词'''
    
​​返回结果​​：

return outputs  # 所有时间步的预测

​​3️⃣ Teacher Forcing 机制​​


​​作用​​：

​​训练阶段​​：以一定概率（teacher_forcing_ratio）使用 ​​真实标签​​（trg[t]）作为解码器的下一个输入，而非模型预测的词。

​​目的​​：加速模型收敛，减少误差累积。

​​流程示例​​：
假设 teacher_forcing_ratio=0.75：

​​75% 的情况​​：输入为真实标签（如 trg[1]）。

​​25% 的情况​​：输入为模型预测的词（如 top1）。

​​代码实现​​：

'''teacher_force = random.random() < teacher_forcing_ratio
input = trg[t] if teacher_force else top1'''

​​4️⃣输出与损失计算​​

​​输出形状​​：[trg_len, batch_size, trg_vocab_size]。

​​损失计算​​：对比每个时间步的预测与真实标签（忽略 <pad>）。

loss = criterion(outputs[1:].view(-1, trg_vocab_size), trg[1:].view(-1))

In [30]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [31]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 编码器初始化
encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)
# 解码器初始化
decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)
# Seq2Seq模型整合
model = Seq2Seq(encoder, decoder, device).to(device)

In [32]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [33]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 13,898,501 trainable parameters


In [34]:
optimizer = optim.Adam(model.parameters())

In [35]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [36]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    # 设置模型为训练模式
    model.train()
    # 初始化当前epoch的总损失
    epoch_loss = 0
    # 遍历数据加载器中的每个批次
    for i, batch in enumerate(data_loader):
        # 获取源语言序列并移动到指定设备
        src = batch["de_ids"].to(device)
        # 获取目标语言序列并移动到指定设备
        trg = batch["en_ids"].to(device)
        # src的形状为 [src length, batch size]
        # trg的形状为 [trg length, batch size]
        
        # 清空优化器的梯度
        optimizer.zero_grad()
        # 将源序列和目标序列输入模型，获取输出
        output = model(src, trg, teacher_forcing_ratio)
        # output的形状为 [trg length, batch size, trg vocab size]
        
        # 获取目标词汇表的大小
        output_dim = output.shape[-1]
        # 去掉输出的第一个时间步（<sos>），并调整形状以适配损失函数
        output = output[1:].view(-1, output_dim)
        # output的形状为 [(trg length - 1) * batch size, trg vocab size]
        
        # 去掉目标序列的第一个时间步（<sos>），并调整形状以适配损失函数
        trg = trg[1:].view(-1)
        # trg的形状为 [(trg length - 1) * batch size]
        
        # 计算当前批次的损失
        loss = criterion(output, trg)
        # 反向传播计算梯度
        loss.backward()
        # 对梯度进行裁剪，防止梯度爆炸
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        # 更新模型参数
        optimizer.step()
        # 累加当前批次的损失
        epoch_loss += loss.item()
    
    # 返回当前epoch的平均损失
    return epoch_loss / len(data_loader)



In [37]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [38]:
n_epochs = 1 # 因模型训练对计算资源要求较高，此处只设立了一轮训练。
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [04:59<00:00, 299.49s/it]

	Train Loss:   5.026 | Train PPL: 152.277
	Valid Loss:   4.861 | Valid PPL: 129.150


In [39]:
model.load_state_dict(torch.load("tut1-model.pt"))

<All keys matched successfully>

In [40]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = de_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [41]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.',
 'A man in an orange hat starring at something.')

In [42]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [43]:
translation

['<sos>',
 'a',
 'man',
 'in',
 'a',
 'a',
 'shirt',
 'is',
 'a',
 'a',
 'a',
 '.',
 '<eos>']